<a href="https://colab.research.google.com/github/mosesj1706/global-electronics-eda/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/mosesj1706/global-electronics-eda.git
%cd global-electronics-eda


Cloning into 'global-electronics-eda'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 3), reused 24 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 1.06 MiB | 4.42 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/global-electronics-eda


In [ ]:
from google.colab import files
import shutil, os

uploaded = files.upload()

for fname in uploaded.keys():
    dst = os.path.join("Uncleaned_CSV_files", fname)
    if os.path.exists(dst):
        os.remove(dst)   # overwrite old version
    shutil.move(fname, dst)

print("✅ Files updated in Uncleaned_CSV_files/")


✅ Files updated in Uncleaned_CSV_files/


In [5]:
# ===============================================
# 📊 Global Electronics - Data Cleaning Notebook
# ===============================================
# Run this notebook in Google Colab.
# Raw CSVs -> Uncleaned_CSV_files/
# Cleaned CSVs -> Cleaned_CSV_files/

# ---- STEP 1: Install dependencies ----
!pip install pandas numpy matplotlib seaborn --quiet

# ---- STEP 2: Import libraries ----
import pandas as pd
import numpy as np
import os

# ---- STEP 3: Project path ----
project_path = "/content/global-electronics-eda"

uncleaned_path = os.path.join(project_path, "Uncleaned_CSV_files")
cleaned_path   = os.path.join(project_path, "Cleaned_CSV_files")

os.makedirs(cleaned_path, exist_ok=True)

print("📂 Uncleaned Path:", uncleaned_path)
print("📂 Cleaned Path:", cleaned_path)

# ---- STEP 4: Load datasets with encoding fallback ----
def load_csv(filename):
    path = os.path.join(uncleaned_path, filename)
    try:
        df = pd.read_csv(path, low_memory=False, encoding="utf-8")
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(path, low_memory=False, encoding="ISO-8859-1")
            print(f"⚠️ Used ISO-8859-1 encoding for {filename}")
        except Exception as e:
            print(f"❌ Error reading {filename}: {e}")
            return None
    print(f"✅ Loaded {filename} | Shape: {df.shape}")
    return df

customers = load_csv("Customers.csv")
products  = load_csv("Products.csv")
sales     = load_csv("Sales.csv")
stores    = load_csv("Stores.csv")
rates     = load_csv("Exchange_Rates.csv")

# ---- STEP 5: Clean Customers ----
if customers is not None:
    customers['Birthday'] = pd.to_datetime(customers['Birthday'], errors='coerce')
    ref_date = pd.to_datetime("today")
    customers['Age'] = ((ref_date - customers['Birthday']).dt.days // 365).astype("Int64")
    bins = [0,18,25,35,45,55,65,120]
    labels = ["<18","18-24","25-34","35-44","45-54","55-64","65+"]
    customers['AgeGroup'] = pd.cut(customers['Age'], bins=bins, labels=labels, right=False)
    customers = customers.drop_duplicates(subset=['CustomerKey'])
    customers.to_csv(os.path.join(cleaned_path,"Customers_clean.csv"), index=False)
    print("✅ Customers cleaned")

# ---- STEP 6: Clean Products ----
if products is not None:
    products['Unit Cost USD'] = pd.to_numeric(products['Unit Cost USD'], errors='coerce')
    products['Unit Price USD'] = pd.to_numeric(products['Unit Price USD'], errors='coerce')
    products['MarginUSD'] = products['Unit Price USD'] - products['Unit Cost USD']
    products = products.drop_duplicates(subset=['ProductKey'])
    products.to_csv(os.path.join(cleaned_path,"Products_clean.csv"), index=False)
    print("✅ Products cleaned")

# ---- STEP 7: Clean Sales ----
if sales is not None:
    sales['Order Date'] = pd.to_datetime(sales['Order Date'], errors='coerce')
    sales['Delivery Date'] = pd.to_datetime(sales['Delivery Date'], errors='coerce')
    sales['Quantity'] = pd.to_numeric(sales['Quantity'], errors='coerce').fillna(0).astype(int)
    sales = sales.drop_duplicates(subset=['Order Number','Line Item'])
    sales.to_csv(os.path.join(cleaned_path,"Sales_clean.csv"), index=False)
    print("✅ Sales cleaned")

# ---- STEP 8: Clean Stores ----
if stores is not None:
    stores['Open Date'] = pd.to_datetime(stores['Open Date'], errors='coerce')
    stores['Square Meters'] = pd.to_numeric(stores['Square Meters'], errors='coerce')
    stores = stores.drop_duplicates(subset=['StoreKey'])
    stores.to_csv(os.path.join(cleaned_path,"Stores_clean.csv"), index=False)
    print("✅ Stores cleaned")

# ---- STEP 9: Clean Exchange Rates ----
if rates is not None:
    rates['Date'] = pd.to_datetime(rates['Date'], errors='coerce')
    rates['Exchange'] = pd.to_numeric(rates['Exchange'], errors='coerce')
    rates = rates.drop_duplicates()
    rates.to_csv(os.path.join(cleaned_path,"Exchange_Rates_clean.csv"), index=False)
    print("✅ Exchange Rates cleaned")

# ---- STEP 10: Verify outputs ----
print("\n📂 Cleaned files created:")
!ls -lh Cleaned_CSV_files


📂 Uncleaned Path: /content/global-electronics-eda/Uncleaned_CSV_files
📂 Cleaned Path: /content/global-electronics-eda/Cleaned_CSV_files
⚠️ Used ISO-8859-1 encoding for Customers.csv
✅ Loaded Customers.csv | Shape: (15266, 10)
✅ Loaded Products.csv | Shape: (2517, 10)
✅ Loaded Sales.csv | Shape: (62884, 9)
✅ Loaded Stores.csv | Shape: (67, 5)
✅ Loaded Exchange_Rates.csv | Shape: (11215, 3)
✅ Customers cleaned
✅ Products cleaned
✅ Sales cleaned
✅ Stores cleaned
✅ Exchange Rates cleaned

📂 Cleaned files created:
total 4.7M
-rw-r--r-- 1 root root 1.5M Sep 23 17:18 Customers_clean.csv
-rw-r--r-- 1 root root 234K Sep 23 17:18 Exchange_Rates_clean.csv
-rw-r--r-- 1 root root 256K Sep 23 17:18 Products_clean.csv
-rw-r--r-- 1 root root  491 Sep 23 17:03 README.md
-rw-r--r-- 1 root root 2.7M Sep 23 17:18 Sales_clean.csv
-rw-r--r-- 1 root root 2.9K Sep 23 17:18 Stores_clean.csv
